In [1]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam, SGD, Nadam, RMSprop
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
# Load weights of convolution layers
from keras import backend as K
vgg_model = VGGFace(include_top=False,weights='vggface', input_shape=(200,200,3))
print('CNN Weights Loaded.')

CNN Weights Loaded.


In [3]:
#Training FC layers while freezing remaining CNN layers
nb_class = 4
hidden_dim = 512

last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

# Finetuning: only the last 3 fully connected layers are trained. The rest are made non-trainable.
layer_count = 0
for layer in custom_vgg_model.layers:
    layer_count = layer_count+1
for l in range(layer_count-3):
    custom_vgg_model.layers[l].trainable=False
    
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 200, 200, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 200, 200, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 100, 100, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 100, 100, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 100, 100, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 50, 50, 128)       0         
__________

In [4]:
# Loading train, test and validation data
train_data_dir = '/Users/seba/desktop/Datasets/CroppedYaleDataset/train'
validation_data_dir = '/Users/seba/desktop/Datasets/CroppedYaleDataset/validate'
#test_data_dir = '/Users/seba/Desktop/YaleDataset/test'
nb_train_samples = 263
nb_validation_samples = 28
nb_test_samples = 22
img_height = 200
img_width = 200 
batch_size = 8

In [5]:
MEAN_VALUE = np.array([103.939, 116.779, 123.68])   # BGR
def preprocess(img):
    # img is (channels, height, width), values are 0-255
    img = img[::-1]  # switch to BGR
    img -= MEAN_VALUE
    return img

In [6]:
# prepare data augmentation configuration
validate_datagen = ImageDataGenerator(
    preprocessing_function=preprocess,
    rescale=1./255,  ######
    featurewise_center=True,
    featurewise_std_normalization=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess,
    rescale=1./255,
    featurewise_center=True, 
    featurewise_std_normalization=True,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 1052 images belonging to 4 classes.
Found 127 images belonging to 4 classes.


In [7]:
# Compile model 
custom_vgg_model.compile(optimizer=SGD(lr=0.01, momentum=0.7, decay=1e-4), loss='categorical_crossentropy',metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,patience=0, min_lr=1e-6)
tensorboard_callback=TensorBoard(log_dir='/Users/seba/Desktop/Logs/log_11/', histogram_freq=0,write_graph=True,write_images=False)


In [8]:
weights_path = '/Users/seba/desktop/Weights/weights_11.h5'
checkpointer = ModelCheckpoint(filepath=weights_path, verbose=1,save_best_only = True, monitor = 'val_loss')


In [9]:
# Finetuning: train last 3 fully connected layers
custom_vgg_model.fit_generator( 
    train_generator,
    steps_per_epoch=nb_train_samples/batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/batch_size,callbacks=[checkpointer,tensorboard_callback])

Epoch 1/30


/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:514: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


32/32 [==============================] - 254s - loss: 1.3791 - acc: 0.2695 - val_loss: 1.3528 - val_acc: 0.2083
Epoch 2/30
32/32 [==============================] - 250s - loss: 1.3425 - acc: 0.4766 - val_loss: 1.2681 - val_acc: 0.7826
Epoch 3/30
32/32 [==============================] - 250s - loss: 1.2916 - acc: 0.6016 - val_loss: 1.1886 - val_acc: 0.6250
Epoch 4/30
32/32 [==============================] - 247s - loss: 1.1918 - acc: 0.7188 - val_loss: 0.9484 - val_acc: 1.0000
Epoch 5/30
32/32 [==============================] - 242s - loss: 1.0613 - acc: 0.8439 - val_loss: 0.8204 - val_acc: 0.9583
Epoch 6/30
32/32 [==============================] - 241s - loss: 0.8840 - acc: 0.8867 - val_loss: 0.5610 - val_acc: 1.0000
Epoch 7/30
32/32 [==============================] - 241s - loss: 0.7096 - acc: 0.9062 - val_loss: 0.4038 - val_acc: 0.9565
Epoch 8/30
32/32 [==============================] - 242s - loss: 0.5580 - acc: 0.9453 - val_loss: 0.2181 - val_acc: 1.0000
Epoch 9/30
32/32 [=========

In [ ]:
# Test network on testing data


In [ ]:
# Train last FC layer or train whole network including CNN layers? 
# Validation_steps??
# Subtract mean of images?? 


#The number of samples processed for each epoch is batch_size * steps_per_epochs.
#steps_per_epoch = number of times weights are updated per epoch